## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-29-16-50-23 +0000,bbc,Mahmood demands migrants earn right to settlem...,https://www.bbc.com/news/articles/c0m4g3zvy02o...
1,2025-09-29-16-46-38 +0000,nypost,"58 million pounds of corn dogs, sausage-on-a-s...",https://nypost.com/2025/09/29/business/58m-pou...
2,2025-09-29-16-46-27 +0000,nyt,Live Updates: Netanyahu and Trump Meet to Disc...,https://www.nytimes.com/live/2025/09/29/world/...
3,2025-09-29-16-46-25 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/29/world/...
4,2025-09-29-16-46-17 +0000,nyt,Investigators Seek Motive for Michigan Church ...,https://www.nytimes.com/2025/09/29/us/michigan...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2344/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,41
72,new,14
30,michigan,13
31,church,12
215,shutdown,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
235,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,117
195,2025-09-29-01-58-54 +0000,nypost,President Trump shows off new White House blin...,https://nypost.com/2025/09/28/us-news/presiden...,89
45,2025-09-29-15-31-00 +0000,wsj,President Trump said films made outside the U....,https://www.wsj.com/politics/elections/trump-r...,81
70,2025-09-29-14-35-40 +0000,wapo,Shutdown nears as Trump plans to meet with lea...,https://www.washingtonpost.com/business/2025/0...,77
53,2025-09-29-15-22-21 +0000,nyt,"New Retail Leaders Confront Trump’s Tariffs, P...",https://www.nytimes.com/2025/09/27/business/tr...,75


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
235,117,2025-09-28-20-43-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
69,69,2025-09-29-14-37-00 +0000,wsj,"Eric Adams Ends Re-Election Bid, Reshaping Rac...",https://www.wsj.com/us-news/new-york-city-mayo...
84,52,2025-09-29-13-58-52 +0000,bbc,Death toll rises to four after Michigan church...,https://www.bbc.com/news/articles/ceq2vd15glwo...
102,35,2025-09-29-12-30-52 +0000,nypost,Netanyahu teases immunity for Hamas in propose...,https://nypost.com/2025/09/29/world-news/netan...
59,34,2025-09-29-15-18-08 +0000,nypost,Video-game giant EA going private in $55B deal...,https://nypost.com/2025/09/29/business/ea-goin...
45,34,2025-09-29-15-31-00 +0000,wsj,President Trump said films made outside the U....,https://www.wsj.com/politics/elections/trump-r...
75,34,2025-09-29-14-24-27 +0000,nypost,"Hurricane Humberto, Tropical Storm Imelda set ...",https://nypost.com/2025/09/29/world-news/hurri...
167,34,2025-09-29-09-00-00 +0000,wsj,"Rahm Emanuel, former Democratic congressman, W...",https://www.wsj.com/politics/policy/rahm-emanu...
109,32,2025-09-29-12-05-00 +0000,wsj,Moldova’s Pro-Europe Party Claims Victory Over...,https://www.wsj.com/world/europe/moldovas-pro-...
119,31,2025-09-29-11-25-31 +0000,nypost,Teen runaway was likely dead for ‘several week...,https://nypost.com/2025/09/29/us-news/teen-run...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
